[#242](https://github.com/chengsoonong/crowdastro/issues/242)

In [1]:
# Some setup.
import h5py, numpy, matplotlib.pyplot as plt, astropy.io.ascii as asc, scipy.spatial, sklearn.model_selection
import crowdastro.crowd.util, sklearn.linear_model, sklearn.ensemble
%matplotlib inline

In [19]:
with h5py.File('/Users/alger/data/Crowdastro/swire.h5', 'r') as f:
    swire_features = f['features'].value
with h5py.File('/Users/alger/data/Crowdastro/crowdastro-swire.h5', 'r') as f:
    swire_coords = f['/swire/cdfs/numeric'][:, :2]
    swire_names = [i.decode('ascii') for i in f['/swire/cdfs/string'].value]
table = asc.read('/Users/alger/data/Crowdastro/one-table-to-rule-them-all.tbl')
swire_tree = scipy.spatial.KDTree(swire_coords)
labels = asc.read('/Users/alger/data/SWIRE/all_labels.csv')
atlas_coords = numpy.array([(r['Component RA (Franzen)'], r['Component DEC (Franzen)']) for r in table
                            if r['Component RA (Franzen)']])

## Generate training sets

We are looking for five different training sets for each subset of the ATLAS-CDFS data:

- RGZ & Norris & compact (the "clean" set)
- RGZ & Norris & resolved
- RGZ & Norris
- RGZ compact
- RGZ resolved
- RGZ

First, let's have a function that takes a set of ATLAS objects and returns five sets of SWIRE objects.

In [10]:
def atlas_to_swire(atlas: list, radius: float=1 / 60) -> list:
    # atlas is a list of table Keys.
    # Look up the coordinates of the ATLAS objects.
    atlas = set(atlas)
    ras = [r['Component RA (Franzen)'] for r in table if r['Key'] in atlas]
    decs = [r['Component DEC (Franzen)'] for r in table if r['Key'] in atlas]
    coords = numpy.vstack([ras, decs]).T
    nearby = sorted({int(i)
                     for i in numpy.concatenate(swire_tree.query_ball_point(coords, radius))})
    return nearby

Next, we'll split ATLAS into three (overlapping) subsets:

- RGZ
- Norris
- Compact

From these we can compute all subsets we want to train on.

In [11]:
rgz = {r['Key'] for r in table if r['Component Zooniverse ID (RGZ)'] and
                                         r['Component ID (Franzen)'] == r['Primary Component ID (RGZ)'] and
                                  r['Component ID (Franzen)']}
norris = {r['Key'] for r in table if r['Component # (Norris)'] and r['Component ID (Franzen)']}
compact = {r['Key'] for r in table if r['Component ID (Franzen)'] and
                                             r['Component S (Franzen)'] / r['Component Sp (Franzen)'] <= 1}

We can now compute the training sets. We will split CDFS in four. Let's start by finding the min/max RA/dec to get our dividing lines.

In [12]:
middle = (numpy.median(atlas_coords[:, 0]), numpy.median(atlas_coords[:, 1]))
middle = (52.8, -28.1)

In [13]:
subsets = [
    ('RGZ & Norris & compact', rgz & norris & compact),
    ('RGZ & Norris & resolved', rgz & norris - compact),
    ('RGZ & Norris', rgz & norris),
    ('RGZ & compact', rgz & compact),
    ('RGZ & resolved', rgz - compact),
    ('RGZ', rgz),
]

In [14]:
training_testing_atlas_sets = {s:[] for s, _ in subsets}  # Maps subset string -> [(train, test)]

def filter_subset(subset: set, q: int) -> set:
    """Filters subset to just include indices of ATLAS objects in a given quadrant."""
    subset_ = set()
    for s in subset:
        row = table[table['Key'] == s][0]
        coords = row['Component RA (Franzen)'], row['Component DEC (Franzen)']

        if (
                (q == 0 and coords[0] >= middle[0] and coords[1] >= middle[1]) or
                (q == 1 and coords[0] < middle[0] and coords[1] >= middle[1]) or
                (q == 2 and coords[0] < middle[0] and coords[1] < middle[1]) or
                (q == 3 and coords[0] >= middle[0] and coords[1] < middle[1])):
            subset_.add(s)
    return subset_

for subset_str, subset_set in subsets:
    for q in range(4):  # Quadrants.
        test = filter_subset(subset_set, q)
        train = {i for i in subset_set if i not in test}
        print(subset_str, len(train), len(test))
        training_testing_atlas_sets[subset_str].append((train, test))

RGZ & Norris & compact 244 143
RGZ & Norris & compact 294 93
RGZ & Norris & compact 329 58
RGZ & Norris & compact 294 93
RGZ & Norris & resolved 123 35
RGZ & Norris & resolved 113 45
RGZ & Norris & resolved 120 38
RGZ & Norris & resolved 118 40
RGZ & Norris 367 178
RGZ & Norris 407 138
RGZ & Norris 449 96
RGZ & Norris 412 133
RGZ & compact 1786 398
RGZ & compact 1547 637
RGZ & compact 1642 542
RGZ & compact 1577 607
RGZ & resolved 221 55
RGZ & resolved 200 76
RGZ & resolved 206 70
RGZ & resolved 201 75
RGZ 2007 453
RGZ 1747 713
RGZ 1848 612
RGZ 1778 682


These can be converted into SWIRE sets.

In [16]:
training_testing_swire_sets = {s:[] for s, _ in subsets}  # Maps subset string -> [(train, test)]

for subset_str, subset_set in subsets:
    for train, test in training_testing_atlas_sets[subset_str]:
        train = atlas_to_swire(train)
        test = atlas_to_swire(test)
        print(subset_str, len(set(train) & set(test)), 'out of', len(set(test)), 'overlap')
        train = sorted(set(train) - set(test))
        training_testing_swire_sets[subset_str].append((train, test))

RGZ & Norris & compact 0 out of 3064 overlap
RGZ & Norris & compact 0 out of 2154 overlap
RGZ & Norris & compact 15 out of 1468 overlap
RGZ & Norris & compact 15 out of 2152 overlap
RGZ & Norris & resolved 0 out of 927 overlap
RGZ & Norris & resolved 0 out of 1079 overlap
RGZ & Norris & resolved 0 out of 1074 overlap
RGZ & Norris & resolved 0 out of 1075 overlap
RGZ & Norris 0 out of 3880 overlap
RGZ & Norris 8 out of 3110 overlap
RGZ & Norris 23 out of 2435 overlap
RGZ & Norris 15 out of 3107 overlap
RGZ & compact 33 out of 7301 overlap
RGZ & compact 84 out of 10877 overlap
RGZ & compact 129 out of 9123 overlap
RGZ & compact 78 out of 10897 overlap
RGZ & resolved 0 out of 1414 overlap
RGZ & resolved 0 out of 1760 overlap
RGZ & resolved 0 out of 1804 overlap
RGZ & resolved 0 out of 1900 overlap
RGZ 65 out of 8317 overlap
RGZ 143 out of 11913 overlap
RGZ 188 out of 10299 overlap
RGZ 110 out of 11976 overlap


## Extracting labels

In [20]:
swire_name_to_rgz_label = {}
swire_name_to_norris_label = {}
for row in labels:
    swire_name_to_norris_label[row['swire']] = bool(row['norris_label']) and row['norris_label'] == 'True'
    swire_name_to_rgz_label[row['swire']] = bool(row['rgz_label']) and row['rgz_label'] == 'True'

## Training LR with Norris labels

We will now train logistic regression on the Norris label set. We will test on both RGZ and Norris, since there are objects that RGZ observes that Norris does not.

In [32]:
def test_on_sets(training_testing_swire_sets, name_to_label, Classifier):
    for subset_str, splits in training_testing_swire_sets.items():
        if 'Norris' not in subset_str:
            continue

        rgz_accuracies = []
        norris_accuracies = []
        for train, test in splits:
            train_features = swire_features[train]
            train_labels = [name_to_label[swire_names[i]] for i in train]

            test_features = swire_features[test]

            lr = Classifier()
            lr.fit(train_features, train_labels)
            
            if 'Norris' in subset_str:
                test_labels_norris = [swire_name_to_norris_label[swire_names[i]] for i in test]
                norris_accuracies.append(
                    crowdastro.crowd.util.balanced_accuracy(test_labels_norris, lr.predict(test_features)))
            test_labels_rgz = [swire_name_to_rgz_label[swire_names[i]] for i in test]
            rgz_accuracies.append(
                crowdastro.crowd.util.balanced_accuracy(test_labels_rgz, lr.predict(test_features)))
        yield subset_str, (rgz_accuracies, norris_accuracies)

In [33]:
for subset, (rgz_acc, norris_acc) in test_on_sets(
        training_testing_swire_sets, swire_name_to_norris_label,
        lambda: sklearn.linear_model.LogisticRegression(class_weight='balanced', penalty='l2', C=1e10)):
    print('{} on RGZ: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(rgz_acc) * 100, numpy.std(rgz_acc) * 100))
    print('{} on Norris: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(norris_acc) * 100, numpy.std(norris_acc) * 100))

RGZ & Norris & compact on RGZ: (77.63 +- 2.10)%
RGZ & Norris & compact on Norris: (94.74 +- 1.40)%
RGZ & Norris & resolved on RGZ: (82.43 +- 2.09)%
RGZ & Norris & resolved on Norris: (89.85 +- 1.65)%
RGZ & Norris on RGZ: (84.44 +- 0.62)%
RGZ & Norris on Norris: (94.44 +- 1.46)%


## Training RF with Norris labels

In [35]:
for subset, (rgz_acc, norris_acc) in test_on_sets(
        training_testing_swire_sets, swire_name_to_norris_label,
        lambda: sklearn.ensemble.RandomForestClassifier(class_weight='balanced')):
    print('{} on RGZ: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(rgz_acc) * 100, numpy.std(rgz_acc) * 100))
    print('{} on Norris: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(norris_acc) * 100, numpy.std(norris_acc) * 100))

RGZ & Norris & compact on RGZ: (71.36 +- 3.11)%
RGZ & Norris & compact on Norris: (88.16 +- 6.47)%
RGZ & Norris & resolved on RGZ: (67.33 +- 1.36)%
RGZ & Norris & resolved on Norris: (79.37 +- 4.33)%
RGZ & Norris on RGZ: (70.96 +- 2.59)%
RGZ & Norris on Norris: (87.51 +- 6.29)%


## Training LR with RGZ labels

In [36]:
for subset, (rgz_acc, norris_acc) in test_on_sets(
        training_testing_swire_sets, swire_name_to_rgz_label,
        lambda: sklearn.linear_model.LogisticRegression(class_weight='balanced', penalty='l2', C=1e10)):
    print('{} on RGZ: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(rgz_acc) * 100, numpy.std(rgz_acc) * 100))
    print('{} on Norris: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(norris_acc) * 100, numpy.std(norris_acc) * 100))

RGZ & Norris & compact on RGZ: (86.95 +- 1.28)%
RGZ & Norris & compact on Norris: (95.07 +- 0.68)%
RGZ & Norris & resolved on RGZ: (81.74 +- 1.37)%
RGZ & Norris & resolved on Norris: (87.84 +- 4.50)%
RGZ & Norris on RGZ: (84.19 +- 0.42)%
RGZ & Norris on Norris: (91.94 +- 1.09)%


## Training RF with RGZ labels

In [38]:
for subset, (rgz_acc, norris_acc) in test_on_sets(
        training_testing_swire_sets, swire_name_to_rgz_label,
        lambda: sklearn.ensemble.RandomForestClassifier(class_weight='balanced')):
    print('{} on RGZ: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(rgz_acc) * 100, numpy.std(rgz_acc) * 100))
    print('{} on Norris: ({:.02f} +- {:.02f})%'.format(
        subset, numpy.mean(norris_acc) * 100, numpy.std(norris_acc) * 100))

RGZ & Norris & compact on RGZ: (79.73 +- 4.20)%
RGZ & Norris & compact on Norris: (88.78 +- 4.53)%
RGZ & Norris & resolved on RGZ: (70.96 +- 8.17)%
RGZ & Norris & resolved on Norris: (76.52 +- 12.86)%
RGZ & Norris on RGZ: (78.45 +- 4.55)%
RGZ & Norris on Norris: (86.07 +- 6.08)%
